In [1]:
from mm_neural_adjoint import Network
import torch
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

In [2]:
X = pd.read_csv('data/data_x_tiny.csv', header=None, delimiter=' ')
y = pd.read_csv('data/data_y_tiny.csv', header=None, delimiter=' ')

X_tensor = torch.tensor(X.values, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.float32)

# Neural Adjoint Training Example

This example demonstrates how to train and evaluate a Neural Adjoint model using the MM-Neural-Adjoint package.

## Data Preparation

First, we create a dataset and split it into training, validation, and test sets:

```python
dataset = TensorDataset(X_tensor, y_tensor)

# Split with 70-15-15 ratio
total_size = len(dataset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [train_size, val_size, test_size]
)
```

## DataLoader Configuration

We create three DataLoaders with different batch sizes:
- Training: batch size 10 with shuffling
- Validation: batch size 10 with shuffling
- Testing: batch size 1 (required for geometry evaluation) with shuffling

```python
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=10, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)
```

## Model Setup

Initialize the Neural Adjoint model with:
- 8 input features (geometry dimensions)
- 300 output features (spectrum dimensions)
- Automatic device selection (GPU if available, otherwise CPU)

```python
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Network(8, 300, device=device)
```

## Training

Train the model for 100 epochs with progress tracking:
- Uses tqdm for progress visualization
- Saves the best model based on validation loss
- Automatically logs metrics using MLflow

```python
EPOCHS = 100
with tqdm(total=EPOCHS, desc='Training Progress') as pbar:
    model.train(EPOCHS, train_loader, val_loader, progress_bar=pbar, save=True)
```

## Evaluation

Finally, evaluate the model's geometry prediction capabilities:
- Uses the test dataset with batch size 1
- Performs inverse design to predict geometries from spectra

```python
model.evaluate_geometry(test_loader)
```

Note: Make sure your input tensors `X_tensor` and `y_tensor` are properly prepared before running this code.

In [3]:
dataset = TensorDataset(X_tensor, y_tensor)

# Split the dataset into train, validation, and test
total_size = len(dataset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [train_size, val_size, test_size]
)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=10, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Network(8, 300, device=device)

EPOCHS = 100

with tqdm(total=EPOCHS, desc='Training Progress') as pbar:
    model.train(EPOCHS, train_loader, val_loader, progress_bar=pbar, save=True)


model.evaluate_geometry(test_loader)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Network(8, 300, device=device)


model.load('checkpoints/best_model.pt')
print(model.geometry_lower_bound)

Successfully loaded model from checkpoints/best_model.pt
tensor([-1., -1., -1., -1., -1., -1., -1., -1.])


In [8]:
spectra = model.predict_spectra(X_tensor[0].unsqueeze(0))
print(spectra.shape)

(1, 300)


In [9]:
Xpred_top, Ypred_top, MSE_top = model.predict_geometry(y_tensor[0])
print(Xpred_top.shape, Ypred_top.shape, MSE_top.shape)


AttributeError: 'tuple' object has no attribute 'shape'